# Procesamiento de Datos de Kaggle con Python
Este notebook descarga y transforma datos del dataset de retrasos y cancelaciones de vuelos, generando archivos en formato Parquet, ORC y Avro.

In [ ]:
# Instalar bibliotecas necesarias
!pip install pandas pyarrow pyorc fastavro kaggle

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyorc
from fastavro import writer, parse_schema
import os

In [ ]:
dataset_url = "https://www.kaggle.com/datasets/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018"
file_name = "2009.csv"
year = "2009"

## Descarga del dataset desde Kaggle
Asegúrate de configurar la API de Kaggle y colocar tu archivo `kaggle.json` en la carpeta `~/.kaggle/` antes de ejecutar esta celda.

In [10]:
# Descargar el dataset desde Kaggle
!kaggle datasets download -d yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018 --unzip
print("Dataset descargado y descomprimido.")

Dataset URL: https://www.kaggle.com/datasets/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018
License(s): other

Dataset descargado y descomprimido.



  0%|          | 0.00/1.95G [00:00<?, ?B/s]
  0%|          | 1.00M/1.95G [00:00<16:51, 2.06MB/s]
  0%|          | 3.00M/1.95G [00:00<06:10, 5.63MB/s]
  0%|          | 6.00M/1.95G [00:00<03:10, 11.0MB/s]
  1%|          | 10.0M/1.95G [00:00<01:54, 18.2MB/s]
  1%|          | 14.0M/1.95G [00:00<01:26, 23.9MB/s]
  1%|          | 18.0M/1.95G [00:01<01:15, 27.3MB/s]
  1%|          | 22.0M/1.95G [00:01<01:10, 29.5MB/s]
  1%|▏         | 26.0M/1.95G [00:01<01:06, 30.9MB/s]
  2%|▏         | 30.0M/1.95G [00:01<01:05, 31.5MB/s]
  2%|▏         | 34.0M/1.95G [00:01<01:07, 30.5MB/s]
  2%|▏         | 38.0M/1.95G [00:01<01:07, 30.2MB/s]
  2%|▏         | 42.0M/1.95G [00:01<01:05, 31.0MB/s]
  2%|▏         | 46.0M/1.95G [00:02<01:03, 32.2MB/s]
  3%|▎         | 50.0M/1.95G [00:02<01:01, 32.9MB/s]
  3%|▎         | 54.0M/1.95G [00:02<01:12, 27.9MB/s]
  3%|▎         | 58.0M/1.95G [00:02<01:07, 29.9MB/s]
  3%|▎         | 62.0M/1.95G [00:02<01:05, 30.9MB/s]
  3%|▎         | 66.0M/1.95G [00:02<01:11, 28.4MB/s]
 

## Cargar el archivo CSV y procesarlo

In [13]:
# Cargar los datos en un DataFrame
df = pd.read_csv(file_name)
print("Datos cargados correctamente.")
df.head()

Datos cargados correctamente.


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,...,62.0,68.0,42.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,...,82.0,75.0,43.0,213.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,...,70.0,62.0,36.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,...,77.0,56.0,37.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,...,105.0,77.0,40.0,213.0,NaN,NaN,NaN,NaN,NaN,NaN


## Guardar el archivo en formato Parquet

In [14]:
# Guardar el archivo en formato Parquet
parquet_file = f"air{year}.parquet"
df.to_parquet(parquet_file, engine="pyarrow", index=False)
print(f"Archivo {parquet_file} guardado.")

Archivo air2009.parquet guardado.


## Guardar el archivo en formato ORC

In [ ]:

import pyarrow as pa
import pyarrow.orc as orc
 
data = pd.read_csv(file_name)
table = pa.Table.from_pandas(data)

with pa.OSFile('air2011.orc', 'wb') as sink:
    orc.write_table(table, sink)

## Guardar el archivo en formato ORC comprimido con Snappy

In [28]:

with pa.OSFile('air2011_snappy.orc', 'wb') as sink:
    orc.write_table(table, sink, compression='snappy')

## Guardar el archivo en formato Avro con campos específicos

In [23]:
# Guardar el archivo en formato Avro
avro_file = f"air{year}_small.avro"
avro_schema = {
    "type": "record",
    "name": "FlightDelay",
    "fields": [
        {"name": "FL_DATE", "type": "string"},
        {"name": "OP_CARRIER", "type": "string"},
        {"name": "DEP_DELAY", "type": ["null", "float"]},  # DEP_DELAY puede ser null
    ],
}

records = df[["FL_DATE", "OP_CARRIER", "DEP_DELAY"]].to_dict(orient="records")
with open(avro_file, "wb") as avro_fp:
    writer(avro_fp, parse_schema(avro_schema), records)
print(f"Archivo {avro_file} guardado.")

Archivo air2009_small.avro guardado.
